In [2]:
import glob
import os
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms

In [3]:
#GPU設定
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

In [4]:
#random seed
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

入力画像の前処理を行う

In [1]:
class ImageTransform():
    
    #constructor
    def __init__(self, resize, mean, std):
        
        self.data_transform = {
            #data augumation
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                resize, scale=(0.5,1.0)),
                transforms.RandomHorizontalFlip(),
                #convert to tensor 
                transforms.ToTensor(),
                #標準化
                transforms.Normalize(mean, std)        
            ]),
            'validation': transforms.Compose([
                transforms.RandomResizedCrop(
                resize, scale=(0.5,1.0)),
                transforms.RandomHorizontalFlip(),
                #convert to tensor 
                transforms.ToTensor(),
                #標準化
                transforms.Normalize(mean, std)        
            ])
        }
    
    def __call__(self, img, phase='train'):
        #前処理モードを指定
        return self.data_transform[phase](img)

画像フォルダパスの作成

In [6]:
def make_datapath_list(phase='train'):
    
    path = './data/'
    target_path = os.path.join(path+phase+'/**/*.jpg')
    print(target_path)
    
    path_list = []
    
    for path in glob.glob(target_path):
        path_list.append(path)
        
    return path_list

train_list = make_datapath_list(phase='train')
validation_list = make_datapath_list(phase='validation')

train_list

./data/train/**/*.jpg
./data/validation/**/*.jpg


['./data/train/crack/cam0137336_9.jpg',
 './data/train/crack/cam1337265_9.jpg',
 './data/train/crack/cam1337333_13.jpg',
 './data/train/crack/cam0137338_7.jpg',
 './data/train/crack/cam0137338_4.jpg',
 './data/train/crack/cam1337338_21.jpg',
 './data/train/crack/cam0137315_10.jpg',
 './data/train/crack/cam0137337_6.jpg',
 './data/train/crack/cam0237284_1.jpg',
 './data/train/crack/cam0137265_7.jpg',
 './data/train/crack/cam0237264_3.jpg',
 './data/train/crack/cam1337261_8.jpg',
 './data/train/crack/cam0237278_1.jpg',
 './data/train/crack/cam0137335_1.jpg',
 './data/train/crack/cam0137279_4.jpg',
 './data/train/crack/cam1337339_11.jpg',
 './data/train/crack/cam1337338_13.jpg',
 './data/train/crack/cam1337337_8.jpg',
 './data/train/crack/cam0337281_2.jpg',
 './data/train/crack/cam1337337_20.jpg',
 './data/train/crack/cam1337261_2.jpg',
 './data/train/crack/cam1337267_4.jpg',
 './data/train/crack/cam0237319_2.jpg',
 './data/train/crack/cam1337263_11.jpg',
 './data/train/crack/cam1337338_3

各クラスごとにデータセットを作成

In [23]:
class HymenopterDataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train', lis=[]):
        #set filelist
        self.file_list = file_list
        #前処理クラスのインスタンス
        self.transform = transform
        #train or validation
        self.phase = phase
        self.classes_list = lis
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        
        img_path = self.file_list[index]
        #print(img_path)
        #PIL Image
        img = Image.open(img_path)
        
        #画像の前処理
        img_transformed = self.transform(img, self.phase)
        
        #label name
        #label nameのコードを使いやすいように改変
        if self.phase == 'train':
            path_lis = img_path.split('/')
            categorical_label = path_lis[3]
        elif self.phase == 'validation':
            path_lis = img_path.split('/')
            categorical_label = path_lis[3]
            
        self.classes_list = ['crack', 'black']
        if not path_lis[3] in self.classes_list:
            self.classes_list.append(categorical_label)
        print(self.classes_list)
        
        label = self.classes_list.index(categorical_label)
        print('label : %d'%label)
        
        return img_transformed, label
 
size = 50
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopterDataset(
                file_list=train_list,
                transform=ImageTransform(size, mean, std),
                phase='train',lis=[])

validation_dataset = HymenopterDataset(
                    file_list=validation_list,
                    transform=ImageTransform(size, mean, std),
                    phase='validation',lis=[])

# 動作確認
#index = 0
for i in range(len(train_list)):
    print(train_dataset.__getitem__(i)[0].size())
    #print(train_dataset.__getitem__(index)[1])


['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([3, 50, 50])
['crack']
label : 0
torch.Size([

DataLoaderの作成

In [20]:
batch_size = 32

#generate date-loader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {'train': train_dataloader, 'val': validation_dataloader}

#iterator ??
#batch 32ごとに格納
batch_iterator = iter(dataloaders_dict['train'])
inputs,labels = next(batch_iterator)
print(inputs.size())

['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 1
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 1
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 1
['crack', 'black']
label : 0
['crack', 'black']
label : 0
['crack', 'black']
label : 1
['crack', 'black']
label : 0
['crack', 'black']
label : 0
torch.Size([32, 3, 50, 50])


ネットワークモデルの作成

In [9]:
#load VGG-16 model
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

#output layer -> label num: 2
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()
print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


損失関数の定義

In [10]:
criterion = nn.CrossEntropyLoss()

最適化オプティマイザの設定

In [11]:
#転移学習にて学習させるパラメータ
params_to_update = []

#学習パラメータ
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        #print(name)
    else:
        param.requires_grad = False


      # params_to_updateの中身を確認
print("-----------")
print(params_to_update)

-----------
[Parameter containing:
tensor([[ 0.0006,  0.0057,  0.0025,  ..., -0.0080,  0.0038, -0.0048],
        [ 0.0035,  0.0092, -0.0013,  ...,  0.0123, -0.0151, -0.0030]],
       requires_grad=True), Parameter containing:
tensor([-0.0013,  0.0078], requires_grad=True)]


In [12]:
# 最適化手法の設定
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

訓練

In [13]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測
                    
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イタレーション結果の計算
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)  
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [14]:
# 学習・検証を実行する
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/2
-------------


  9%|▉         | 1/11 [00:00<00:01,  5.51it/s]

val Loss: 1.0946 Acc: 0.0966
Epoch 2/2
-------------


  0%|          | 0/5 [00:00<?, ?it/s]

train Loss: 0.2112 Acc: 0.8688


100%|██████████| 5/5 [00:00<00:00,  5.25it/s]

val Loss: 0.0338 Acc: 0.9793
